In [1]:
import os
import textwrap
from dotenv import load_dotenv, find_dotenv
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain import PromptTemplate

In [2]:
load_dotenv(find_dotenv())
embeddings = OpenAIEmbeddings()

## Elegir si enseñar o recordar (evita el uso de tokens en OpenAI)

### 1. Enseñar

In [3]:
# Carga de documentos y extracción de información
# (asegúrate de que haya PDFs en la carpeta documentos)
documents = DirectoryLoader('./documentos/', glob="./*.pdf", loader_cls=PyPDFLoader).load()
# Tratameinto de caracteres indeseados
for d in documents:
    d.page_content = d.page_content.replace('\n', ' ').replace('\t', ' ')
# Separador de texto (límita el texto para que sea soportado por el LLM)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200
)
textos = text_splitter.split_documents(documents)
# Se instruye (creación de un knowledge-base) con la info
# proporcionada
persist_directory = input('¿Cómo se llama este knoledge-base?: ') + '_kb'
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=textos, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)
# Se guarda el conocimiento generado en el directorio actual
# para ser usado sin tener que instruir al LLM
vectordb.persist()
vectordb = None
os.system(f'zip -r db.zip ./{persist_directory}')

c:\Users\luisr\OneDrive\Documentos\GitHub\1day_1thing\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


1

### 2. Recordar (si y solo si Enseñar=Done)

In [4]:
# Now we can load the persisted database from disk, and use it as normal. 
persist_directory = input('¿Qué knoledge-base deseas usar?: ') + '_kb'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)
retriever = vectordb.as_retriever(search_kwargs={"k": 5})
#retriever = vectordb.as_retriever()

In [5]:
# Se especifica el LLM de OpenAI
turbo_llm = ChatOpenAI(
    temperature=0.7,
    model_name='gpt-3.5-turbo-0125'
)
# Se personaliza el LLM
template = """
Eres Prometeo, un asistente personal de lectura que habla Español.

Tu tarea consiste en:

1. Ser carismático y ofrecer información sobre ti y tus funciones.

2. Leer detalladamente la información proporcionada en documentos
de texto en formato PDF, para ser capaz de dar respuestas a cualquier tipo de pregunta,
en especial a preguntas puntuales.

SIEMPRE debes responder con base al contexto proporcionado aqui: {context}.
        
Si no sabes la respuesta, puedes decir: 'No sé'.

NUNCA hables del contexto.

Teniendo lo anterior en cuenta, responde la siguiente pregunta: {question}
"""

prometeo_prompt = PromptTemplate(
    template=template, input_variables=["context", "question"]
)
# Creación de cadena
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm,
                                       chain_type="stuff",
                                       retriever=retriever,
                                       chain_type_kwargs={"prompt": prometeo_prompt},
                                       return_source_documents=True)
# Formato y referencias de las respuestas
def wrap_text_preserve_newlines(text, width=70):
    lines = text.split('\n')
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    wrapped_text = '\n'.join(wrapped_lines)
    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\nReferencias:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [6]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = qa_chain(query)
process_llm_response(llm_response)

¿Cuáles son los enfoques usados en la investigación?


c:\Users\luisr\OneDrive\Documentos\GitHub\1day_1thing\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Los enfoques utilizados en la investigación son el enfoque cualitativo
y el enfoque cuantitativo.

Referencias:
documentos\4.pdf
documentos\4.pdf
documentos\4.pdf
documentos\4.pdf
documentos\4.pdf


In [7]:
#Demo break down
query = query#input("Hazme una pregunta: ")
llm_response = qa_chain(query)
llm_response

{'query': '¿Cuáles son los enfoques usados en la investigación?',
 'result': 'Los enfoques utilizados en la investigación son cualitativo y cuantitativo.',
 'source_documents': [Document(page_content='De acuerdo con la selección y uso de  las técnicas metodológicas, se evidenció entre  las principales seleccionadas, la entrevista,  puesto que su aplicación prevaleció en los  estudios en Europa: El primero fue realizado  en Grecia, en donde hicieron uso de la técnica  de la entrevista a expertos para indagar acerca  de los problemas de desarrollo de innovación,  falta de fondos y la comercialización de  productos (Pechlivanidis, 1993). Por otra  parte, también fue utilizada la entrevista en  estudios realizados en Reino Unido y en la  Unión Europea, buscando el contexto político  y el alcance de mejoras de programas públicos  basados en programas de la universidad que permitían el mejoramiento en innovación  (Vega et al., 2012). En África, se realizaron entrevistas a  los innovadores in

In [8]:
# Demo
query = input("Hazme una pregunta: ")
print(query)
llm_response = qa_chain(query)
process_llm_response(llm_response)

¿Quién o quiénes son los autores del documento?
Los autores del documento son Rodríguez-Burgos, Karla y Miranda-
Medina, Carlos F.

Referencias:
documentos\4.pdf
documentos\4.pdf
documentos\4.pdf
documentos\4.pdf
documentos\4.pdf


In [9]:
#Demo break down
query = query#input("Hazme una pregunta: ")
llm_response = qa_chain(query)
llm_response

{'query': '¿Quién o quiénes son los autores del documento?',
 'result': 'Los autores del documento son Rodríguez-Burgos, Karla y Miranda-Medina, Carlos F.',
 'source_documents': [Document(page_content='Elaboración propia, 2023 en base a 1.091 artículos provenientes de la base de datos Scopus  y  exportados al software VOSviewer . Figura I:  Asociación de investigaciones de innovación en las pyme con otras  palabras clave Lo antes expuesto destaca que es  importante hacer un análisis que describa  ampliamente el conocimiento que se genera  sobre la innovación en la pyme, derivado de la información que disponen las publicaciones  científicas, patentes y los productos industriales  que representan de una forma más cercana a  la realidad, los resultados de investigación', metadata={'page': 11, 'source': 'documentos\\4.pdf'}),
  Document(page_content='110Licencia de Creative Commons  Atribución 4.0 Internacional (CC BY 4.0) https://creativecommons.org/licenses/by/4.0/deed.es   Rodríguez-Bur

In [11]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)


Eres Prometeo, un asistente personal de lectura que habla Español.

Tu tarea consiste en:

1. Ser carismático y ofrecer información sobre ti y tus funciones.

2. Leer detalladamente la información proporcionada en documentos
de texto en formato PDF, para ser capaz de dar respuestas a cualquier tipo de pregunta,
en especial a preguntas puntuales.

SIEMPRE debes responder con base al contexto proporcionado aqui: {context}.
        
Si no sabes la respuesta, puedes decir: 'No sé'.

NUNCA hables del contexto.

Teniendo lo anterior en cuenta, responde la siguiente pregunta: {question}

